I was inspired by the literature "Juding LLM-as-a-Judge with MT-Bench and Chatbot Arena". I would like to use this method to test the Chinese text generation of GPT-3.5 and text it with ERNIE-3.5

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


Getting the generated text from gpt-3.5 Turbo

In [11]:
import http.client
import json

theme_list = ["自由","生命","理想","高山","大海"]
from openai import OpenAI
text_list = []
client = OpenAI(
    # This is the default and can be omitted
    api_key = "sk-9qxqayMe8D42SRZ3r4yb4SRjdF0Xd2FwUmavZUJZXSTC6VnJ",
    base_url = "https://api.chatanywhere.tech/v1"
)
for theme in theme_list:
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              f"content": "请围绕{theme}这个主题写一首七言绝句,并且生成的内容只需要包括诗歌主体",
          }
      ],
      model="gpt-3.5-turbo",
  )
  text = chat_completion.choices[0].message.content
  text_list.append(text)
print(text_list)

['乘风破浪展回帆，心中主题永不变。\n奋力向前再出发，追寻梦想与希望。', '春风柔软抚青枝，\n映入窗帘梦正醒。\n花香扑鼻溢轻袖，\n心灵荡漾一片情。', '诗歌主体：\n追寻人生的辉煌道，\n落英飞雁乐章悠。\n探索自我在宇宙，\n艰辛涤荡意识流。\n\n命运离奇如繁星，\n争突映照心灵舟。\n人生如戏剧性展，\n创造璀璨人世炼。', '深秋的枫叶飘红落，\n远山的云彩飞扬翻。\n清心自在在山水，\n诗意长存在人间。', '春风吹过万物苏，\n花开芬芳色香浮。\n阳光洒满湖泊畔，\n映照彩云翩翩舞。']


Using ERNIE-3.5 to evaluate the quality of the generated text

In [32]:
api_key = "vk05T4TAIxYy6TU6Pk0c9hMz"
secret_key = "zGlgT0MScaGtvQ1lcUEGRLFEjgvGm08U"
import pandas

import requests
import json

def get_access_token():
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """

    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"

    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")
rate = {}
for text in  text_list:
  url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()

  payload = json.dumps({
      "messages": [
          {
              "role": "user",
              "content": f"你的回答只需要一个数字。请你用以下标准：（评分等级为1到5）：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性；③相关性。生成的唐诗与给定主题是否相关；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等；⑤整体。从整体情况评价生成的唐诗。诗歌为:{text}"
          }
      ],
      "response_format":"json_object"
  })
  headers = {
      'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  print(response.text)

{"id":"as-qckexqyjtf","object":"chat.completion","created":1711804486,"result":"{\n    \"格式\": 3,\n    \"创新性\": 3,\n    \"相关性\": 4,\n    \"美学\": 4,\n    \"整体\": 4\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":130,"completion_tokens":43,"total_tokens":173}}
{"id":"as-h4kstjyfha","object":"chat.completion","created":1711804492,"result":"{\n    \"格式\": 3,\n    \"创新性\": 3,\n    \"相关性\": 3,\n    \"美学\": 3,\n    \"整体\": 3\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":135,"completion_tokens":43,"total_tokens":178}}
{"id":"as-wi6t14jb0t","object":"chat.completion","created":1711804496,"result":"{\n    \"格式\": 4,\n    \"创新性\": 3,\n    \"相关性\": 4,\n    \"美学\": 4,\n    \"整体\": 4\n}","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":169,"completion_tokens":43,"total_tokens":212}}
{"id":"as-t3sab5ymuc","object":"chat.completion","created":17